In [1]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torchvision import transforms

In [2]:
train_data_path = '/opt/ml/code/df/train/df_train_15.csv'
valid_data_path = '/opt/ml/code/df/valid/df_valid_15.csv'

In [3]:
df_train = pd.read_csv(train_data_path)
df_valid = pd.read_csv(valid_data_path)

In [ ]:
def preprocess(df):
    

In [6]:
del df['race']
del df['id']
del df['age']
df['gender'] = df['gender'].map({'male':0, 'female':1})

In [7]:
df

,gender,path
0,1,000001_female_Asian_45
1,1,000002_female_Asian_52
2,0,000004_male_Asian_54
3,1,000005_female_Asian_58
4,1,000006_female_Asian_59
...,...,...
2695,0,006954_male_Asian_19
2696,0,006955_male_Asian_19
2697,0,006956_male_Asian_19
2698,0,006957_male_Asian_20


In [8]:
df['mask'] = [['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal'] for _ in range(len(df))]

In [9]:
df_train, df_valid = train_test_split(df, test_size=0.25, random_state=42, stratify=df.gender)

In [10]:
def generate_mask_field(df):
    
    def _get_extension(row):
        for f in os.listdir(os.path.join(train_data_path, 'images', row.path)):
            if f.startswith(row['mask']):
                return os.path.join(train_data_path, 'images', row.path, f)
        raise FileNotFoundError

    def _label(row):
        return row['gender'] + 2*row['mask']
    
    mask_map = {'incorrect_mask': 1, 'mask1': 0, 'mask2': 0, 'mask3': 0,
                'mask4': 0, 'mask5': 0, 'normal': 2}    

    df = df.explode('mask')
    df['path'] = df.apply(_get_extension, axis=1)
    df['mask'] = df['mask'].map(mask_map)
    df['label'] = df.apply(_label, axis=1)

    return df

In [48]:
df.to_csv('df_all.csv', index=False)

In [11]:
df_train = generate_mask_field(df_train)
df_valid = generate_mask_field(df_valid)

In [13]:
df_train.to_csv('df/df_mask_gender_train.csv', index=False)
df_valid.to_csv('df/df_mask_gender_valid.csv', index=False)